In [6]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

In [2]:
IMAGE_SIZE = 256
BATCH_SIZE = 32
CHANNELS = 3
EPOCHS = 35
n_classes = 10

tf.config.get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "Tomatoes",
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
)

Found 16011 files belonging to 10 classes.


In [4]:
class_names = dataset.class_names
print(class_names)

['Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']


In [5]:
def get_dataset_partitions_tf(
    ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000
):
    ds_size = len(ds)
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)

    if shuffle:
        ds.shuffle(shuffle_size, seed=22)

    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size + val_size)

    return train_ds, val_ds, test_ds

In [6]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
# caches images for next epoch, prefetch fetches next batch before next iteration

In [7]:
resize_and_rescale = tf.keras.Sequential(
    [
        layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
        layers.experimental.preprocessing.Rescaling(1.0 / 255),
    ]
)

In [8]:
data_augmentation = tf.keras.Sequential(
        [
            layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
            layers.experimental.preprocessing.RandomRotation(0.2),
        ]
    )

In [9]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)

In [10]:
model = models.Sequential(
    [
        resize_and_rescale,
        data_augmentation,
        layers.Conv2D(32, (3, 3), activation="relu", input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(
            64,
            (3, 3),
            activation="relu",
            padding="same",
        ),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(
            64,
            (3, 3),
            activation="relu",
            padding="same",
        ),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(
            64,
            (3, 3),
            activation="relu",
            padding="same",
        ),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(
            64,
            (3, 3),
            activation="relu",
            padding="same",
        ),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(
            64,
            (3, 3),
            activation="relu",
            padding="same",
        ),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(
            64,
            (3, 3),
            activation="relu",
            padding="same",
        ),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dense(n_classes, activation="softmax"),
    ]
)

In [11]:
model.build(input_shape=input_shape)

In [12]:
model.compile(
    optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=["accuracy"],
)

In [13]:
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1,
    validation_data=val_ds,
)

Epoch 1/35
400/400 [==============================] - 95s 224ms/step - loss: 1.9711 - accuracy: 0.2840 - val_loss: 1.6809 - val_accuracy: 0.3988
Epoch 2/35
400/400 [==============================] - 93s 232ms/step - loss: 1.5148 - accuracy: 0.4748 - val_loss: 1.4688 - val_accuracy: 0.5031
Epoch 3/35
400/400 [==============================] - 92s 230ms/step - loss: 1.1543 - accuracy: 0.5982 - val_loss: 1.3809 - val_accuracy: 0.5481
Epoch 4/35
400/400 [==============================] - 86s 216ms/step - loss: 1.0040 - accuracy: 0.6534 - val_loss: 1.9319 - val_accuracy: 0.4681
Epoch 5/35
400/400 [==============================] - 87s 217ms/step - loss: 0.9225 - accuracy: 0.6784 - val_loss: 1.9861 - val_accuracy: 0.4794
Epoch 6/35
400/400 [==============================] - 87s 217ms/step - loss: 0.9876 - accuracy: 0.6588 - val_loss: 1.9857 - val_accuracy: 0.5063
Epoch 7/35
400/400 [==============================] - 86s 216ms/step - loss: 0.8502 - accuracy: 0.7087 - val_loss: 1.9567 - val_ac

In [20]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [14]:
scores = model.evaluate(test_ds)

51/51 [==============================] - 5s 45ms/step - loss: 2.4142 - accuracy: 0.6189


In [15]:
model.save("../model/2")

INFO:tensorflow:Assets written to: ../model/2/assets


INFO:tensorflow:Assets written to: ../model/2/assets
